In [ ]:
import pandas as pd
import numpy as np

In [ ]:
doc = pd.read_csv("news.csv")

In [ ]:
doc.head()

In [ ]:
companies = pd.read_csv('/Users/lyd/Downloads/constituents-financials_csv.csv')
company_list = pd.DataFrame(companies)
company_list.index=company_list['Symbol']
company_list = company_list[['Symbol','Name','Sector']]
company_names = company_list['Name'].values
company_symbols = company_list['Symbol'].values
company_description = company_list[['Symbol','Name','Name']].values

In [ ]:
def parse_company_name(company_description):
    stop_words=['&',".",'the','company','inc','inc.','plc','corp','corp.','co','co.','worldwide','corporation','group','']
    for k in range(2):
        for i in range(len(company_description)):
            chars = company_description[i][2].split(" ")

            for j in chars:
                if j.lower() in stop_words:
                    chars.remove(j)
            company_description[i][2] = ' '.join(chars)
    return company_description.tolist()


In [ ]:
company_name_list = parse_company_name(company_description)
company_name_list

In [ ]:
stock_prices = pd.read_csv("price.csv")
stock_prices = stock_prices.set_index(pd.DatetimeIndex(stock_prices['timestamp']))

In [ ]:
stock_prices["timestamp"] = pd.to_datetime(stock_prices["timestamp"], format="%Y-%m-%d")
doc["date"] = pd.to_datetime(doc["date"], format="%Y-%m-%d")

In [ ]:
from pandas.tseries.offsets import BDay
isBusinessDay = BDay().onOffset
match_series = pd.to_datetime(doc['date']).map(isBusinessDay)
news = doc[match_series]
news_data = news.dropna()

In [ ]:
import nltk
import string
import spacy
import re
from nltk.tokenize import sent_tokenize,word_tokenize
from datetime import datetime, timedelta

nlp = spacy.load('en_core_web_sm')
def extract_company_and_price_data(news,date,company_name_list):
    
    stop_words = nltk.corpus.stopwords.words('english')
    stop_words.extend(list(string.ascii_lowercase))
    stop_words.append('would')
    stop_words.append('kmh')
    stop_words.append('mph')
    stop_words.append('u')
    
    date_symbols=['JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG','SEP','OCT','NOV','DEC','MON','TUE','WED','THU','FRI','SAT','SUN']
    
    processed_data=[]
    extracted_information = []
    #regular_expressions = re.compile(r'[^A-Za-z-]')
    
    try:
        sentences = sent_tokenize(news)

        for sentence in sentences:
            tokens = word_tokenize(sentence)
            sentence_en = nlp(sentence)
            extracted_entities = list(map(str,sentence_en.ents))
            bigrams = list(nltk.bigrams(tokens))


            for company in company_name_list:
                if company[0] in stock_prices:
                    if company[0] in extracted_entities \
                    or company[1] in extracted_entities \
                    or company[2] in sentence:
                        future_price = pd.to_numeric(stock_prices[stock_prices["timestamp"] == date][company[0]]).tolist()[0]
                        if (date.weekday() == 0): #monday
                            past_price_1 = pd.to_numeric(stock_prices[stock_prices["timestamp"] == date - timedelta(days=3)][company[0]]).tolist()
                            past_price_2 = pd.to_numeric(stock_prices[stock_prices["timestamp"] == date - timedelta(days=4)][company[0]]).tolist()
                            past_price_3 = pd.to_numeric(stock_prices[stock_prices["timestamp"] == date - timedelta(days=5)][company[0]]).tolist()

                        elif (date.weekday() == 1): #tuesday
                            past_price_1 = pd.to_numeric(stock_prices[stock_prices["timestamp"] == date - timedelta(days=1)][company[0]]).tolist()
                            past_price_2 = pd.to_numeric(stock_prices[stock_prices["timestamp"] == date - timedelta(days=4)][company[0]]).tolist()
                            past_price_3 = pd.to_numeric(stock_prices[stock_prices["timestamp"] == date - timedelta(days=5)][company[0]]).tolist()

                        if (date.weekday() == 2): #wednesday
                            past_price_1 = pd.to_numeric(stock_prices[stock_prices["timestamp"] == date - timedelta(days=1)][company[0]]).tolist()
                            past_price_2 = pd.to_numeric(stock_prices[stock_prices["timestamp"] == date - timedelta(days=2)][company[0]]).tolist()
                            past_price_3 = pd.to_numeric(stock_prices[stock_prices["timestamp"] == date - timedelta(days=5)][company[0]]).tolist()
                        else: #thursday and friday
                            past_price_1 = pd.to_numeric(stock_prices[stock_prices["timestamp"] == date - timedelta(days=1)][company[0]]).tolist()
                            past_price_2 = pd.to_numeric(stock_prices[stock_prices["timestamp"] == date - timedelta(days=2)][company[0]]).tolist()
                            past_price_3 = pd.to_numeric(stock_prices[stock_prices["timestamp"] == date - timedelta(days=3)][company[0]]).tolist()

                        if (len(past_price_1) + len(past_price_2) + len(past_price_3) >= 3):
                            price_movement = np.sign(future_price - past_price_1[0])
                            extracted_information.append((date,sentence,tokens,sentence_en,bigrams,company[0],company[1],company[2],future_price,\
                                                          past_price_1[0],past_price_2[0],past_price_3[0],price_movement))
    except:
        print("something wrong")
    return extracted_information

In [ ]:
a = extract_company_and_price_data(news_data["content"].iloc[100],news_data["date"].iloc[100],company_name_list)
b = extract_company_and_price_data(news_data["content"].iloc[101],news_data["date"].iloc[101],company_name_list)

In [ ]:
a.extend(b)
h = pd.DataFrame(a,columns =['date','sentences','tokens','sentence_en','bigrams','symbol','name','processed_name','future_price','past_price_1day',\
                        'past_price_2day','past_price_3day','price_movement'])
h["bigrams"][0]

In [ ]:
all_information=[]
for i in range(len(news_data)):
    print("processing news no."+ str(i))
    single_day = extract_company_and_price_data(news_data["content"].iloc[i],news_data["date"].iloc[i],company_name_list)
    if (len(single_day) > 0):
        all_information.extend(single_day)